In [1]:
import numpy as np
import pandas as pd
import csv
from mlxtend.preprocessing import TransactionEncoder
import time

## Process CSV file into Nested List

In [2]:
df = pd.read_csv("D:\\393- assignment\\grocery_transactions.csv")
data = pd.DataFrame(df)
data = df
data

,Transaction ID,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7
0,1,cheese,beef,bananas,rice,lettuce,apples,bread
1,2,pasta,potatoes,tomatoes,bananas,beef,lettuce,cheese
2,3,beef,eggs,pork,tomatoes,milk,apples,chicken
3,4,beef,carrots,bread,pasta,tomatoes,milk,onions
4,5,pork,beef,apples,lettuce,eggs,milk,bananas
...,...,...,...,...,...,...,...,...
21995,21996,eggs,pasta,beef,potatoes,rice,tomatoes,onions
21996,21997,pasta,cheese,carrots,onions,lettuce,apples,bread
21997,21998,lettuce,apples,milk,beef,bread,rice,pork
21998,21999,milk,potatoes,chicken,bread,beef,tomatoes,pasta


In [3]:
# Open the CSV file
with open("D:\\393- assignment\\grocery_transactions.csv", newline='') as csvfile:

    # Read the CSV file into a dictionary
    data = csv.DictReader(csvfile)

    # Create an empty list to hold the transactions
    transactions = []

    # Loop through each row in the CSV file
    for row in data:
        
        # Create a list to hold the items in this transaction
        transaction_items = []
        
        # Loop through each column in the row (excluding the first column, which is the transaction ID)
        for column in list(row.keys())[1:]:
            
            # If the item in this column is not empty, add it to the list of items in this transaction
            if row[column] != '':
                transaction_items.append(row[column])
        
        # Add this transaction to the list of transactions
        transactions.append(transaction_items)

# Print the resulting array of arrays
print(transactions)


[['cheese', 'beef', 'bananas', 'rice', 'lettuce', 'apples', 'bread'], ['pasta', 'potatoes', 'tomatoes', 'bananas', 'beef', 'lettuce', 'cheese'], ['beef', 'eggs', 'pork', 'tomatoes', 'milk', 'apples', 'chicken'], ['beef', 'carrots', 'bread', 'pasta', 'tomatoes', 'milk', 'onions'], ['pork', 'beef', 'apples', 'lettuce', 'eggs', 'milk', 'bananas'], ['beef', 'bananas', 'rice', 'tomatoes', 'eggs', 'potatoes', 'onions'], ['eggs', 'chicken', 'potatoes', 'milk', 'tomatoes', 'lettuce', 'bananas'], ['cheese', 'pasta', 'lettuce', 'eggs', 'apples', 'chicken', 'bread'], ['carrots', 'lettuce', 'bread', 'rice', 'beef', 'bananas', 'onions'], ['milk', 'cheese', 'apples', 'tomatoes', 'carrots', 'lettuce', 'bread'], ['chicken', 'cheese', 'rice', 'carrots', 'beef', 'pork', 'pasta'], ['onions', 'chicken', 'lettuce', 'tomatoes', 'pork', 'milk', 'apples'], ['rice', 'eggs', 'pork', 'apples', 'carrots', 'pasta', 'lettuce'], ['onions', 'eggs', 'carrots', 'chicken', 'bananas', 'pasta', 'lettuce'], ['milk', 'rice'

In [4]:
#create matrix visualization of the dataset

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

In [5]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,apples,bananas,beef,bread,carrots,cheese,chicken,eggs,lettuce,milk,onions,pasta,pork,potatoes,rice,tomatoes
0,True,True,True,True,False,True,False,False,True,False,False,False,False,False,True,False
1,False,True,True,False,False,True,False,False,True,False,False,True,False,True,False,True
2,True,False,True,False,False,False,True,True,False,True,False,False,True,False,False,True
3,False,False,True,True,True,False,False,False,False,True,True,True,False,False,False,True
4,True,True,True,False,False,False,False,True,True,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,False,False,True,False,False,False,False,True,False,False,True,True,False,True,True,True
21996,True,False,False,True,True,True,False,False,True,False,True,True,False,False,False,False
21997,True,False,True,True,False,False,False,False,True,True,False,False,True,False,True,False
21998,False,False,True,True,False,False,True,False,False,True,False,True,False,True,False,True


# Apriori Algorithm Implementation

In [6]:
from itertools import combinations

def apriori(df, min_sup=0.5):
    """
    Calculates the frequent itemsets and their support values based on the Apriori algorithm.

    Parameters:
        df (Pandas DataFrame): The input DataFrame containing the transaction data.
        min_sup (float): The minimum support threshold.

    Returns:
        sdf (Pandas DataFrame): A new DataFrame with columns "Itemsets" and "Support", 
        where "Itemsets" contains the comma-separated item names of frequent itemsets 
        and "Support" contains the support rate for each frequent itemset.
    """
    # Initialize an empty list to store the frequent itemsets and their corresponding support values.
    frequent_itemsets = []

    # Initialize a set to store the frequent itemsets generated in the previous iteration.
    prev_itemsets = set()

    # Generate the candidate itemsets using the Apriori algorithm.
    for size in range(1, len(df.columns)+1):

        # Initialize a list to store the itemsets and their support counts.
        itemsets = []

        # Iterate through all possible combinations of columns with the current itemset size.
        for cols in combinations(df, size):

            # Check if all subsets of the current itemset are frequent.
            is_frequent = True
            for subset_size in range(1, size):
                for subset_cols in combinations(cols, subset_size):
                    subset = ",".join(subset_cols)
                    if subset not in prev_itemsets:
                        is_frequent = False
                        break
                if not is_frequent:
                    break

            # If all subsets are frequent, calculate the support count for the current itemset.
            if is_frequent:
                count = df[list(cols)].all(axis=1).sum()
                itemsets.append((cols, count))

        # Pruning - Add the itemsets if they meet the minimum support threshold.
        for cols, count in itemsets:
            support = count / len(df)
            if support >= min_sup:
                frequent_itemsets.append([",".join(cols), support])

        # Update the final table with the frequent itemsets generated in previous iteration.
        prev_itemsets = set([",".join(cols) for cols, count in itemsets])

    # Create a new Pandas DataFrame from the frequent_itemsets list, with columns named "Itemsets" and "Support".
    result_df = pd.DataFrame(frequent_itemsets, columns=["Itemsets", "Support"])

    # Return the new DataFrame.
    return result_df


### my Apriori

In [7]:
#My Apriori runtime
start_time = time.time()
frequent_itemsets = apriori(df, min_sup=0.1)
print("--- Total runtime of my Apriori implementation is: %s seconds ---" % (time.time() - start_time))
frequent_itemsets = frequent_itemsets.sort_values(by='Support', ascending=False)
frequent_itemsets

--- Total runtime of my Apriori implementation is: 0.1917874813079834 seconds ---


,Itemsets,Support
8,lettuce,0.442864
0,apples,0.441500
13,potatoes,0.441364
9,milk,0.440864
1,bananas,0.440182
...,...,...
99,"chicken,tomatoes",0.170091
91,"chicken,eggs",0.170091
70,"carrots,cheese",0.170091
96,"chicken,pork",0.169045


### mlxtend Apriori 

In [8]:
# Mlxtend Apriori Function runtime
start_time = time.time()
from mlxtend.frequent_patterns import apriori
frequent_itemsets_2 = apriori(df, min_support = 0.1, use_colnames=True)
print("--- Total runtime of mlxtend Apriori implementation is: %s seconds ---" % (time.time() - start_time))
frequent_itemsets_2 = frequent_itemsets_2.sort_values(by='support', ascending=False)
frequent_itemsets_2

--- Total runtime of mlxtend Apriori implementation is: 0.09753704071044922 seconds ---


,support,itemsets
8,0.442864,(lettuce)
0,0.441500,(apples)
13,0.441364,(potatoes)
9,0.440864,(milk)
1,0.440182,(bananas)
...,...,...
99,0.170091,"(tomatoes, chicken)"
91,0.170091,"(eggs, chicken)"
70,0.170091,"(carrots, cheese)"
96,0.169045,"(pork, chicken)"


# Association Rule Mining

In [9]:
def association_rules(itemsets, min_conf):
    """
    Given a dataframe of frequent itemsets and a minimum confidence threshold, returns the association rules
    that meet the minimum confidence threshold.
    
    Args:
    - itemsets: a pandas dataframe with two columns: "Itemsets" and "Support", where "Itemsets" contains the frequent itemsets
    and "Support" contains the support values for each frequent itemset
    - min_conf: a float representing the minimum confidence threshold
    
    Returns:
    - a pandas dataframe with three columns: "Antecedent", "Consequent", and "Confidence", where "Antecedent" and "Consequent"
    contain the itemsets in the association rule and "Confidence" contains the confidence value for each association rule
    """
    # Initialize an empty list to store the association rules.
    rules = []
    
    # Store the set of all frequent itemsets in a set for quick lookup.
    freq_itemsets = set(itemsets["Itemsets"].str.split(",").apply(frozenset))
    
    # Iterate through each row of the frequent itemsets DataFrame.
    for _, row in itemsets.iterrows():
        
        # Get the set of items in the current itemset.
        items = set(row["Itemsets"].split(","))
        
        # Iterate through all possible non-empty subsets of the current itemset.
        for i in range(1, len(items)):
            for antecedent in combinations(items, i):
                
                # Compute the consequent as the set difference between the itemset and the antecedent.
                consequent = items.difference(set(antecedent))
                
                # Check if both the antecedent and consequent are present in the frequent itemsets set.
                if frozenset(antecedent) in freq_itemsets and frozenset(consequent) in freq_itemsets:
                    
                    # Calculate the confidence of the association rule.
                    antecedent_support = itemsets[itemsets.Itemsets == ",".join(antecedent)].iloc[0]["Support"]
                    rule_support = row["Support"]
                    rule_confidence = rule_support / antecedent_support
                    
                    # Check if the confidence meets the minimum threshold.
                    if rule_confidence >= min_conf:
                        # Add the association rule to the rules list.
                        rules.append({"Antecedent": ",".join(antecedent), "Consequent": ",".join(consequent), "Confidence": rule_confidence, "Support": rule_support})
    
    # Create a new DataFrame from the rules list.
    rules_df = pd.DataFrame(rules, columns=["Antecedent", "Consequent", "Confidence", "Support"])
    
    # Return the new DataFrame.
    return rules_df



In [10]:
start_time = time.time()
rules = association_rules(frequent_itemsets, min_conf=0.4)
# Sort the result by confidence in descending order
rules_sorted = rules.sort_values(by='Confidence', ascending=False)
print("--- Total runtime of my association rules generator is: %s seconds ---" % (time.time() - start_time))
rules_sorted

--- Total runtime of my association rules generator is: 0.10835599899291992 seconds ---


,Antecedent,Consequent,Confidence,Support
3,cheese,lettuce,0.414505,0.180818
5,beef,milk,0.412302,0.180682
1,bananas,lettuce,0.411710,0.181227
53,chicken,potatoes,0.411460,0.176909
10,carrots,apples,0.409979,0.179273
...,...,...,...,...
119,tomatoes,cheese,0.400063,0.173136
118,tomatoes,bananas,0.400063,0.173136
120,chicken,beef,0.400042,0.172000
78,bananas,milk,0.400041,0.176091


In [28]:
rules = association_rules(frequent_itemsets, min_conf=0.4)
# Sort the result by support in descending order
rules_sorted = rules.sort_values(by='Support', ascending=False)
rules_sorted

,Antecedent,Consequent,Confidence,Support
0,lettuce,bananas,0.409217,0.181227
1,bananas,lettuce,0.411710,0.181227
2,cheese,lettuce,0.414505,0.180818
3,lettuce,cheese,0.408293,0.180818
4,milk,beef,0.409836,0.180682
...,...,...,...,...
116,chicken,cheese,0.403848,0.173636
117,tomatoes,carrots,0.401218,0.173636
118,tomatoes,bananas,0.400063,0.173136
119,tomatoes,cheese,0.400063,0.173136


### mlxtend rule miner

In [11]:
from mlxtend.frequent_patterns import association_rules

start_time = time.time()
result = association_rules(frequent_itemsets_2, metric='confidence', min_threshold=0.4)
#filter out unnecessary information, the final table will only show antecedents, consequents, and their confidence.
result = result[['antecedents', 'consequents', 'confidence']]
# Sort the result by confidence in descending order
result = result.sort_values(by='confidence', ascending=False)
print("--- Total runtime of mlxtend association rules generator is: %s seconds ---" % (time.time() - start_time))
result

--- Total runtime of mlxtend association rules generator is: 0.006086587905883789 seconds ---


,antecedents,consequents,confidence
3,(cheese),(lettuce),0.414505
4,(beef),(milk),0.412302
1,(bananas),(lettuce),0.411710
53,(chicken),(potatoes),0.411460
10,(carrots),(apples),0.409979
...,...,...,...
119,(tomatoes),(cheese),0.400063
118,(tomatoes),(bananas),0.400063
120,(chicken),(beef),0.400042
78,(bananas),(milk),0.400041
